In [17]:
import ngsolve as ngs
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw
from ngsolve.krylovspace import CGSolver
from ttictoc import tic, toc

mesh = ngs.Mesh('whatever.vol')
mesh.Curve(2)
# mesh.Refine()

In [18]:
%run J.ipynb

using 3d mesh with ne= 72204  elements and nv= 12535  points
Solving...
Number of iterations = 503/1000 | Residual = 9.792831543321432e-06
2999.9999999999873
Solving...
Number of iterations = 442/1000 | Residual = 9.899156656591516e-06
3000.0020577247274


In [19]:
from bhdata import BHCurves
fun_dw  = BHCurves(3)
fun_w   = fun_dw.Integrate()
fun_ddw = fun_dw.Differentiate()

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil|ambient|default"
nonlinear = "r_steel|l_steel|mid_steel"
# nonlinear = "mid_steel"

lin = 0

BH data from TEAM 20 problem
returning energy


In [20]:
# print(HCurl.ndof)
# print(mesh.GetMaterials())

HCurl = ngs.HCurl(mesh, order = deg, nograds = True, dirichlet = 'ambient_face')
# HCurl = ngs.HCurl(mesh, order = deg, nograds = True)
u,v = HCurl.TnT()


# Nonlinear:

maxit = 10000
tol2 = 1e-8
regb = 1e-12

A = ngs.GridFunction(HCurl)
B = ngs.curl(A)
normB = ngs.sqrt(B*B + regb)


if lin == 1: cf_energy = mesh.MaterialCF({linear: nu0/2*B*B, nonlinear: nu0/2*B*B}, default = nu0/2*B*B).Compile()
else: cf_energy = mesh.MaterialCF({linear: nu0/2*B*B, nonlinear: fun_w(normB)}, default = nu0/2*B*B).Compile()

def fun_W():
    with ngs.TaskManager(): res = ngs.Integrate(cf_energy - ngs.curl(A)*Hs, mesh, order = 2*deg)
    # with ngs.TaskManager(): res = ngs.Integrate(cf_energy - ngs.curl(Hs)*A, mesh)
    return res

ir = ngs.IntegrationRule(ngs.fem.ET.TET, order = 2*deg)


if lin == 1: cf_rhs = mesh.MaterialCF({linear: nu0, nonlinear: nu0}, default = nu0).Compile()
else: cf_rhs = mesh.MaterialCF({linear: nu0, nonlinear: fun_dw(normB)/normB}, default = nu0).Compile()

rhs = ngs.LinearForm(HCurl)
rhs += ngs.SymbolicLFI(cf_rhs*B*ngs.curl(v) - ngs.curl(v)*Hs, intrule = ir)

# rhs = ngs.LinearForm((cf_rhs*B*ngs.curl(v) - ngs.curl(v)*Hs)*ngs.dx)
# rhs = ngs.LinearForm((cf_rhs*B*ngs.curl(v) - ngs.curl(Hs)*v)*ngs.dx)

def fun_dW(): #implicitly depending on A!
    with ngs.TaskManager(): rhs.Assemble()
    return rhs


Id = ngs.CF((1,0,0,
             0,1,0,
             0,0,1), dims=(3,3))

BBt = ngs.CF((B[0]*B[0], B[0]*B[1], B[0]*B[2],
              B[1]*B[0], B[1]*B[1], B[1]*B[2],
              B[2]*B[0], B[2]*B[1], B[2]*B[2]), dims = (3,3))


fun1 = fun_dw(normB)/normB
fun2 = (fun_ddw(normB) - fun_dw(normB)/normB)/(normB*normB)

if lin == 1: cf_iter = mesh.MaterialCF({linear: nu0*Id, nonlinear: nu0*Id}, default = nu0*Id).Compile()
else: cf_iter = mesh.MaterialCF({linear: nu0*Id, nonlinear: fun1*Id + fun2*BBt}, default = nu0*Id).Compile()

K_iter = ngs.BilinearForm(HCurl)
K_iter += ngs.SymbolicBFI(cf_iter*ngs.curl(u)*ngs.curl(v), intrule = ir)
# K_iter += (cf_iter*ngs.curl(u)*ngs.curl(v))*ngs.dx
C_iter = ngs.Preconditioner(K_iter, type = "local")

def fun_ddW():
    with ngs.TaskManager(): K_iter.Assemble()
    return K_iter


In [21]:
print("Using 3D mesh with ne=", mesh.ne, "elements and nv=", mesh.nv, "points and " ,HCurl.ndof, "DOFs.\n ")

with ngs.TaskManager(): A.Set(ngs.CF((0,0,0)))

du = ngs.GridFunction(HCurl)
uo = ngs.GridFunction(HCurl)
wo = 1e12

for it in range(1,maxit+1):
    tic()
    # with ngs.TaskManager():
    #     K_iter.Assemble()
    #     rhs.Assemble()
    #     res = ngs.Integrate(cf_energy - Hs*ngs.curl(A), mesh)
    
    w  = fun_W()
    dw = fun_dW()
    da = fun_ddW()
    tm1 = toc()
    
    tic()
    # du.vec.data = da.mat.Inverse(HCurl.FreeDofs(), inverse="sparsecholesky") * dw.vec 
    # iterativeSolver = CGSolver(K_iter.mat, freedofs = HCurl.FreeDofs(), atol = 1e-2,  maxiter = maxit, printrates = False)
    with ngs.TaskManager():
        iterativeSolver = CGSolver(K_iter.mat, pre = C_iter.mat, tol  = 1e-8,  maxiter = maxit)
        du.vec.data = iterativeSolver * dw.vec
    
    if len(iterativeSolver.residuals) == maxit: print("... Failure!")
    # print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")
    tm2 = toc()

    nrm = ngs.InnerProduct(du.vec,dw.vec)
    
    if it == 1:
        nrm0 = nrm
    
    # wn = 1e12
    if abs(wo-w) < tol2:
    # if abs(wn-w) < tol2:
    # if nrm/nrm0 < tol2:
        print("converged to desired tolerance")
        break
    # elif abs(wo-w) < tol2*1e-2:
    #     print("stopped early due to stagnation")
    #     break
    else:
        # linesearch
        uo.vec.data = A.vec.data
        wo = w
        alpha = 1
        for init in range(1,21):
            A.vec.data -= alpha*du.vec.data
            wn = fun_W()
            if wn < w - alpha*0.1*nrm:
                print("Iter: %2d | assem : %.2fs | CG took %.2fs with %4d iterations | alpha : %.2f | energy = %.10f | relres = %.2e |"  %(it,tm1,tm2,iterativeSolver.iterations,alpha,w,nrm/nrm0))
                break
            else:
                alpha = alpha/2
                A.vec.data = uo.vec.data

Using 3D mesh with ne= 72204 elements and nv= 12535 points and  374520 DOFs.
 
Iter:  1 | assem : 1.15s | CG took 3.54s with  710 iterations | alpha : 0.50 | energy = 0.0000000000 | relres = 1.00e+00 |
Iter:  2 | assem : 1.11s | CG took 4.22s with  824 iterations | alpha : 0.25 | energy = -1.1462423561 | relres = 6.47e-01 |
Iter:  3 | assem : 1.17s | CG took 3.35s with  635 iterations | alpha : 1.00 | energy = -1.4440580818 | relres = 1.04e-01 |
Iter:  4 | assem : 1.18s | CG took 2.97s with  575 iterations | alpha : 1.00 | energy = -1.5760871824 | relres = 1.04e-02 |
Iter:  5 | assem : 1.22s | CG took 2.75s with  539 iterations | alpha : 1.00 | energy = -1.5918413137 | relres = 3.02e-04 |
Iter:  6 | assem : 1.18s | CG took 2.73s with  531 iterations | alpha : 1.00 | energy = -1.5922809701 | relres = 5.61e-06 |
Iter:  7 | assem : 1.13s | CG took 2.31s with  475 iterations | alpha : 1.00 | energy = -1.5922888203 | relres = 1.38e-08 |
Iter:  8 | assem : 1.16s | CG took 2.26s with  458 ite

In [22]:
scalars = mesh.MaterialCF({"coil":1, "mid_steel": 2}, default = 0)
L2 = ngs.L2(mesh, order = 2)
scal = ngs.GridFunction(L2)
scal.Set(scalars)

vtk = ngs.VTKOutput(mesh, coefs = [B,ngs.curl(Hs),scal],names = ["B","J","scalars"], filename = "B3", order = 1)
vtk.Do()

'B3'

In [23]:
import pyvista as pv
pvmesh = pv.read("B3.vtu")
pvmesh.set_active_scalars("scalars")
pvmesh.set_active_vectors("B")
threshed = pvmesh.threshold([0.8,2.1])
threshed.set_active_vectors("B")

p = pv.Plotter()
p.add_mesh(threshed, style = 'surface', color = "w", opacity = 0.01, label = None)

arrows = pvmesh.glyph(scale = "B", orient = True, tolerance = 0.01, factor = 0.01) #0.8
# arrows = pvmesh.glyph(scale = "B", orient = True, tolerance = 0.01, factor = 0.01)
p.add_mesh(arrows, color = "yellow")

pvmesh.set_active_vectors("J")
arrows2 = pvmesh.glyph(scale = "J", orient = True, tolerance = 0.001, factor = 1*1e-8)
p.add_mesh(arrows2, color = "black")

p.show(jupyter_backend = "html")
# p.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [24]:
print(ngs.Integrate(ngs.Norm(B),mesh))
print(fun_W())

0.0003601645006816109
-1.592288839575335
